In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/My Drive/0-Research-Project/MedStar/MS_CODE/FieldNN


# Introduction


This is the for 

* module `fieldnn.basicnn.reducer` module
* module `fieldnn.configfn.reducerfn` module
* module `fieldnn.basicnn.merger` module
* module `fieldnn.configfn.mergerfn` module

# Batch

In [2]:
import os
from fieldnn.dataset import RFGDataset, my_collate_fn
from torch.utils.data import DataLoader

Tensor_folder = 'data/ProcData/FldGrnTensor/'
recfldgrn_list = ['P@PatEcInfo-InfoGrn',  'P-EC-PNSect@AllText-TknzGrn']
full_recfldgrn_list = ['B-' + i for i in recfldgrn_list]

# from the get_grain_fn to get the Elig_Set.
Elig_Set = ['P4', 'P5', 'P6', 'P7']

dataset = RFGDataset(Tensor_folder, recfldgrn_list, Elig_Set, RecRootID = 'PID')
print(len(dataset), '<---- dataset')
dataloader = DataLoader(dataset, batch_size = 4, shuffle = True, collate_fn = my_collate_fn)
print(len(dataloader), '<---- dataset')


for idx, batch in enumerate(dataloader):
    # print(f'\n------ {idx}')
    batch_rfg, batch_y = batch
    for k, v in batch_rfg.items(): print(k, v.shape)
    break
    # for k, v in batch_rfg.items(): print(k, v.shape)
    # print(batch_y.shape)

4 <---- dataset
1 <---- dataset
B-P@PatEcInfo-InfoGrn_wgt torch.Size([4, 43])
B-P@PatEcInfo-InfoGrn_tknidx torch.Size([4, 43])
B-P@PatEcInfo-InfoGrn_fldidx torch.Size([4, 43])
B-P-EC-PNSect@AllText-TknzGrn_wgt torch.Size([4, 23, 14, 221])
B-P-EC-PNSect@AllText-TknzGrn_tknidx torch.Size([4, 23, 14, 221])
B-P-EC-PNSect@AllText-TknzGrn_fldidx torch.Size([4, 23, 14, 221])


In [3]:
# Prepare the Input
RECFLD_TO_TENSOR = {}
for full_recfldgrn in full_recfldgrn_list:
    RECFLD_TO_TENSOR[full_recfldgrn] = {k: v for k, v in batch_rfg.items() if full_recfldgrn in k}
    
[i for i in RECFLD_TO_TENSOR]

['B-P@PatEcInfo-InfoGrn', 'B-P-EC-PNSect@AllText-TknzGrn']

In [5]:
from fieldnn.dataflowfn.embedflowfn import get_EmbeddingBlock_SubUnit

from fieldnn.dataflowfn.baseflowfn import mapping_SubUnitName_to_SubUnitNNList
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_Default_NNPara_List
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_BasicNN_Config_List

############################################# Hyperparameters
default_BasicNNtype_To_NNName = {
    'expander': None, # will be updated according to the Grn Type
    'reducer': 'Max',
    'merger': 'Merger',
    'learner': None, # TODO: ignore this currently
    
}
#############################################

############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################


default_SubUnitName = 'E'
fldgrn_folder = 'data/ProcData/FldGrnInfo'
learner_default_dict = {} # To update it in the future. 


df_SubUnit = get_EmbeddingBlock_SubUnit(full_recfldgrn_list, default_SubUnitName)

s = df_SubUnit.apply(lambda x: mapping_SubUnitName_to_SubUnitNNList(x['SubUnitName'], 
                                                                    x['input_names'],
                                                                    default_BasicNNtype_To_NNName), 
                    axis = 1)
df_SubUnit['SubUnit_BasicNN_List'] = s
s = df_SubUnit.apply(lambda x: get_SubUnit_Default_NNPara_List(x['SubUnit_BasicNN_List'], 
                                                               x['input_names'],
                                                               fldgrn_folder, 
                                                               learner_default_dict), axis = 1)

df_SubUnit['SubUnit_DefaultBasicNN_List'] = s



s = df_SubUnit.apply(lambda x: get_SubUnit_BasicNN_Config_List(x['SubUnit_BasicNN_List'], 
                                                               x['SubUnit_DefaultBasicNN_List'], 
                                                               x['input_names'], 
                                                               x['output_name'], 
                                                                embed_size, 
                                                                process, 
                                                               ), axis = 1)

df_SubUnit['SubUnit_BasicNN_Config_List'] = s
df_SubUnit

,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List,SubUnit_BasicNN_Config_List
0,E,[B-P@PatEcInfo-InfoGrn],B-P@PatEcInfo-Info,3,[expander-CateEmbed],"[{'full_recfldgrn': 'B-P@PatEcInfo-InfoGrn', '...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
1,E,[B-P-EC-PNSect@AllText-TknzGrn],B-P-EC-PNSect@AllText-Tknz,5,[expander-LLMEmbed],[{'full_recfldgrn': 'B-P-EC-PNSect@AllText-Tkn...,"[{'nn_type_nn_name': 'expander-LLMEmbed', 'Bas..."


In [6]:
from fieldnn.module.embedblock import EmbedBlockLayer

EmbedBlock = EmbedBlockLayer(df_SubUnit)

RECFLD_TO_EMBEDTESNOR = EmbedBlock(RECFLD_TO_TENSOR)

for k, v in RECFLD_TO_EMBEDTESNOR.items():
    print(k, v['info'].shape)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


B-P@PatEcInfo-Info torch.Size([4, 43, 128])
B-P-EC-PNSect@AllText-Tknz torch.Size([4, 23, 14, 221, 128])


# Get Input

In [7]:
for full_recfld, info_dict in RECFLD_TO_EMBEDTESNOR.items():
    print(full_recfld, info_dict['info'].shape)

B-P@PatEcInfo-Info torch.Size([4, 43, 128])
B-P-EC-PNSect@AllText-Tknz torch.Size([4, 23, 14, 221, 128])


# Reducer: BasicNN Name 

## ReducerSum

In [8]:
import torch

class ReduceSumLayer(torch.nn.Module):
    def __init__(self):
        super(ReduceSumLayer, self).__init__()   

    def forward(self, info, leng_mask):
        # (bs, xxx, l, dim) --> (bs, xxx, dim)
        info = torch.sum(info, -2)
        return info

## ReducerMean

In [9]:
import torch

class ReduceMeanLayer(torch.nn.Module):
    def __init__(self):
        super(ReduceMeanLayer, self).__init__()
  
    def forward(self, info, leng_mask):
        leng = (leng_mask == 0).sum(-1).unsqueeze(-1).float()
        leng[leng == 0.] = 1.0 # change pad to any non-zeros to be dominators.
        info = torch.sum(info, -2) # (bs, xxx, l, dim) --> (bs, xxx, dim)
        info = info/leng           # (bs, xxx, dim)    --> (bs, xxx, dim)
        return info


## ReducerMax

In [10]:
import torch

class RecuderMaxLayer(torch.nn.Module):
    def __init__(self):
        super(RecuderMaxLayer, self).__init__()

    def forward(self, info, leng_mask):
        info = info.masked_fill(leng_mask.unsqueeze(-1), -10000) # double check this.
        a, b = info.max(-2) # not necessary, all the values could be smaller than 0.
        return a
    

## ReducerCat

In [11]:

import math

class ConcatenateLayer(torch.nn.Module):
    def __init__(self):
        super(ConcatenateLayer, self).__init__()

    def forward(self, info, leng_mask):
        l, dim = info.shape[-2:]
        info = info.view(*info.shape[:-2],  l*dim)
        return info   


# Reducer: Basic NN Type

## Module

In [12]:
import torch

# from fieldnn.basicnn.reducer.rdcmean import ReduceMeanLayer
# from fieldnn.basicnn.reducer.rdcsum import ReduceSumLayer
# from fieldnn.basicnn.reducer.rdcmax import RecuderMaxLayer
# from fieldnn.basicnn.reducer.rdccat import ConcatenateLayer

# from .rdcmean import ReduceMeanLayer
# from .rdcsum import ReduceSumLayer
# from .rdcmax import RecuderMaxLayer
# from .rdccat import ConcatenateLayer


class Reducer_Layer(torch.nn.Module):
    def __init__(self, input_names_nnlvl, output_name_nnlvl, reducer_layer_para):
        super(Reducer_Layer, self).__init__()
        
        # Part 0: Meta
        # here input_names and out_tensor just the tensor name, 
        # intead, the info_dict contains the corresponding real tensors.
        assert len(input_names_nnlvl) == 1
        self.input_names_nnlvl = input_names_nnlvl
        self.input_name_nnlvl = input_names_nnlvl[0]
        
        # output_name should be generated from the input_names
        self.output_name_nnlvl = output_name_nnlvl
        
        # the input feature dim size and output feature dim size
        self.input_size = reducer_layer_para['input_size']
        self.output_size = reducer_layer_para['output_size']

        # Part 1: NN
        nn_name, nn_para = reducer_layer_para['nn_name'], reducer_layer_para['nn_para']
        if nn_name.lower() == 'mean':
            self.reducer = ReduceMeanLayer()
        elif nn_name.lower() == 'sum':
            self.reducer = ReduceSumLayer()
        elif nn_name.lower() == 'max':
            self.reducer = RecuderMaxLayer()
        elif nn_name.lower() == 'concat':
            self.reducer = ConcatenateLayer()
            # TODO: need to assert something
            assert self.output_size % self.input_size == 0
        else:
            raise ValueError(f'There is no layer for "{nn_name}"')
            
        # Part 2: PostProcess
        self.postprocess = torch.nn.ModuleDict()
        for method, config in reducer_layer_para['postprocess'].items():
            if method == 'activator':
                activator = config
                if activator.lower() == 'relu': 
                    self.postprocess[method] = torch.nn.ReLU()
                elif activator.lower() == 'tanh': 
                    self.postprocess[method] = torch.nn.Tanh()
                elif activator.lower() == 'gelu':
                    self.postprocess[method] = torch.nn.GELU()
            elif method == 'dropout':
                self.postprocess[method] = torch.nn.Dropout(**config)
            elif method == 'layernorm':
                self.postprocess[method] = torch.nn.LayerNorm(self.output_size, **config)
            
    def forward(self, input_names_nnlvl, INPUTS_TO_INFODICT):
        # information preparation.
        # 'INPUTS_TO_INFODICT` will come from SubUnit Layer.
        assert len(input_names_nnlvl) == 1
        input_name_nnlvl = input_names_nnlvl[0]
        assert self.input_name_nnlvl == input_name_nnlvl
        
        info_dict = INPUTS_TO_INFODICT[input_name_nnlvl]
        holder, info = info_dict['holder'], info_dict['info']
        
        # print(holder.shape, info.shape)
        # the following part is the data proprocessing
        leng_mask = holder == 0
        info = self.reducer(info, leng_mask)
        
        for name, layer in self.postprocess.items():
            info = layer(info)
            
        holder = (leng_mask == 0).sum(-1)
        
        # output_name_nnlvl is not necessarily to be stored in the 
        return self.output_name_nnlvl, {'holder': holder, 'info': info}
    

## Config

In [13]:
def get_reducer_para(nn_name, nn_para, input_size, output_size, postprocess):
    reducer_layer_para = {}
    reducer_layer_para['nn_type'] = 'reducer'
    reducer_layer_para['nn_name'] = nn_name
    reducer_layer_para['nn_para'] = nn_para
    reducer_layer_para['input_size'] = input_size
    reducer_layer_para['output_size'] = output_size
    reducer_layer_para['postprocess'] = postprocess
    return reducer_layer_para

# Merger: BasicNN Name

## Merger-Merge

In [14]:
import torch

class MergeLayer(torch.nn.Module):
    def __init__(self):
        super(MergeLayer, self).__init__()
    
    def forward(self, tensor_list, order = -2):
        info = torch.cat([i.unsqueeze(order) for i in tensor_list], order)
        return info   

# Merger: BasicNN Type

## Module

In [15]:
import torch
# from .merge import MergeLayer

class Merger_Layer(torch.nn.Module):
    def __init__(self, input_names_nnlvl, output_name_nnlvl, merger_layer_para):
        super(Merger_Layer, self).__init__()
        
        # the input_names_nnlvl
        self.input_names_nnlvl = input_names_nnlvl
        # output_name should be generated from the input_names
        self.output_name_nnlvl = output_name_nnlvl
        
        self.input_size = merger_layer_para['input_size']
        self.output_size = merger_layer_para['output_size']
        
        # Part 1: NN
        nn_name = merger_layer_para['nn_name']
        nn_para = merger_layer_para['nn_para']
        
        if nn_name.lower() == 'merger':
            self.merger = MergeLayer()
        else:
            raise ValueError(f'The NN "{nn_name}" is not available')
        
        # Part 2: PostProcess
        self.postprocess = torch.nn.ModuleDict()
        for method, config in merger_layer_para['postprocess'].items():
            if method == 'activator':
                activator = config
                if activator.lower() == 'relu': 
                    self.postprocess[method] = torch.nn.ReLU()
                elif activator.lower() == 'tanh': 
                    self.postprocess[method] = torch.nn.Tanh()
                elif activator.lower() == 'gelu':
                    self.postprocess[method] = torch.nn.GELU()
            elif method == 'dropout':
                self.postprocess[method] = torch.nn.Dropout(**config)
            elif method == 'layernorm':
                self.postprocess[method] = torch.nn.LayerNorm(self.output_size, **config)
                
    def forward(self, input_names_nnlvl, INPUTS_TO_INFODICT):
        
        INPUTS = {k:v for k, v in INPUTS_TO_INFODICT.items() if k in input_names_nnlvl}

        # (1) holder
        holder = self.merger([data['holder'] for fld, data in INPUTS.items()], -1)
        
        # (2) merge data
        info = self.merger([data['info'] for fld, data in INPUTS.items()], -2)
        
        # (3) post-process
        for name, layer in self.postprocess.items():
            info = layer(info)

        return self.output_name_nnlvl, {'holder': holder, 'info': info}

## Config

In [16]:
def get_merger_para(nn_name, nn_para, input_size, output_size, postprocess):
    para = {}
    para['nn_type'] = 'merger'
    para['nn_name'] = nn_name
    para['nn_para'] = nn_para
    para['input_size'] = input_size
    para['output_size'] = output_size
    para['postprocess'] = postprocess
    return para